In [ ]:
from shapely.geometry import Polygon

from worldcereal.rdm_api import RdmInteraction

### 1. Query collections

In [ ]:
# Access all public collections
rdm_interaction = RdmInteraction()
ids = rdm_interaction.query_collections()
print(len(ids))
print(ids)

In [ ]:
# Access public collections for crop type
ewoc_codes = [1106000020] # soybean
ids = rdm_interaction.query_collections(ewoc_codes=ewoc_codes)
print(len(ids))
print(ids)

In [ ]:
# Access public collections for geometry (located near Kenya)
coords = [
(34.45619011, -0.91010781),
(34.79638823, -0.91010781),
(34.79638823, -0.34539808),
(34.45619011, -0.34539808),
(34.45619011, -0.91010781)]
polygon = Polygon(coords)   
ids = rdm_interaction.query_collections(geometry=polygon)
print(len(ids))
print(ids)

In [ ]:
# Access public collections for bounding box and year
temporal_extent = ["2020-01-01", "2020-12-31"]
ids = rdm_interaction.query_collections(geometry=polygon,
                                        temporal_extent=temporal_extent)
print(len(ids))
print(ids)

In [ ]:
# Access private collections
    # without authentication
ids = rdm_interaction.query_collections(include_public=False,
                                        include_private=True)
print(len(ids))
print(ids)

In [ ]:
# Access private collections
    # with authentication
rdm_interaction = RdmInteraction().authenticate()
ids = rdm_interaction.query_collections(include_public=False,
                                        include_private=True)
print(len(ids))
print(ids)

In [ ]:
# Access private collections for crop type
ids = rdm_interaction.query_collections(include_public=False,
                                        include_private=True,
                                        ewoc_codes=ewoc_codes)
print(len(ids))
print(ids)

In [ ]:
# Access both public and private collections for crop type
ids = rdm_interaction.query_collections(include_private=True,
                                        ewoc_codes=ewoc_codes)
print(len(ids))
print(ids)